1. Load and Explore the Data

Import the necessary libraries: pandas and sqlite3.
Connect to the IPL database and load the master table to understand the structure.
Load all the tables and print their column names to identify common columns.

In [ ]:
import sqlite3, pandas as pd
conn = sqlite3.connect("/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/database.sqlite")

pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)


,name
0,Ball_by_Ball
1,Batsman_Scored
2,Batting_Style
3,Bowling_Style
4,City
5,Country
6,Extra_Runs
7,Extra_Type
8,Match
9,Out_Type


In [15]:
pd.read_sql_query("PRAGMA table_info('Player_Match');", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Match_Id,INTEGER,1,None,1
1,1,Player_Id,INTEGER,1,None,2
2,2,Role_Id,INTEGER,1,None,0
3,3,Team_Id,INTEGER,0,None,0


In [16]:
pd.read_sql_query("PRAGMA table_info('sqlite_sequence');", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,name,,0,None,0
1,1,seq,,0,None,0


pd.read_sql_query("PRAGMA table_info('Player_Match');", conn)

In [12]:

pd.read_sql_query("PRAGMA table_info('Player_Match');", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Match_Id,INTEGER,1,None,1
1,1,Player_Id,INTEGER,1,None,2
2,2,Role_Id,INTEGER,1,None,0
3,3,Team_Id,INTEGER,0,None,0


2. Query 1: Select All Columns from Player’s Table

Write and execute a SQL query to select all columns from the Player_Match table.


In [14]:
sql='''

SELECT * 
FROM Player_Match
'''


df_sql = pd.read_sql_query(sql,con=conn)
df_sql.head()


,Match_Id,Player_Id,Role_Id,Team_Id
0,335987,1,1,1
1,335987,2,3,1
2,335987,3,3,1
3,335987,4,3,1
4,335987,5,3,1


3. Query 2: Batsman vs Runs

Write and execute a SQL query to calculate the total runs scored by each batsman.

In [17]:
sql='''
WITH balls AS (
  SELECT b.Match_Id, b.Over_Id, b.Ball_Id, b.Innings_No, b.Striker
  FROM Ball_by_Ball b
),
runs AS (
  SELECT bs.Match_Id, bs.Over_Id, bs.Ball_Id, bs.Innings_No, bs.Runs_Scored
  FROM Batsman_Scored bs
)
SELECT p.Player_Id,
       p.Player_Name,
       SUM(r.Runs_Scored) AS total_runs
FROM balls b
JOIN runs r
  ON r.Match_Id=b.Match_Id
 AND r.Over_Id=b.Over_Id
 AND r.Ball_Id=b.Ball_Id
 AND r.Innings_No=b.Innings_No
JOIN Player p
  ON p.Player_Id=b.Striker
GROUP BY p.Player_Id, p.Player_Name
ORDER BY total_runs DESC;
'''


df_sql = pd.read_sql_query(sql,con=conn)
df_sql.head()

,Player_Id,Player_Name,total_runs
0,21,SK Raina,4106
1,8,V Kohli,4105
2,57,RG Sharma,3874
3,40,G Gambhir,3634
4,162,CH Gayle,3447


4. Query 3: Fifties and Hundreds

Write and execute a SQL query to calculate the number of fifties and hundreds scored by each batsman.

In [18]:
sql='''
WITH batsman_runs_per_match AS (
  SELECT 
    p.Player_Id,
    p.Player_Name,
    b.Match_Id,
    SUM(bs.Runs_Scored) AS runs_in_match
  FROM Ball_by_Ball b
  JOIN Batsman_Scored bs
    ON bs.Match_Id=b.Match_Id
   AND bs.Over_Id=b.Over_Id
   AND bs.Ball_Id=b.Ball_Id
   AND bs.Innings_No=b.Innings_No
  JOIN Player p
    ON p.Player_Id = b.Striker
  GROUP BY p.Player_Id, p.Player_Name, b.Match_Id
)
SELECT
  Player_Id,
  Player_Name,
  SUM(CASE WHEN runs_in_match BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS fifties,
  SUM(CASE WHEN runs_in_match >= 100 THEN 1 ELSE 0 END)           AS hundreds
FROM batsman_runs_per_match
GROUP BY Player_Id, Player_Name
ORDER BY hundreds DESC, fifties DESC, Player_Name;
'''

df_sql = pd.read_sql_query(sql,con=conn)
df_sql.head()

,Player_Id,Player_Name,fifties,hundreds
0,162,CH Gayle,21,5
1,8,V Kohli,27,4
2,110,AB de Villiers,22,3
3,187,DA Warner,32,2
4,41,V Sehwag,16,2


5. Query 4: Best Bowling Figures

Write and execute a SQL query to find the best bowling figures for each bowler.

In [19]:
sql='''
WITH bowling_balls AS (
  SELECT Match_Id, Over_Id, Ball_Id, Innings_No, Bowler
  FROM Ball_by_Ball
),
runs_conceded AS (
  SELECT 
    bb.Bowler AS bowler_id,
    bb.Match_Id,
    SUM(bs.Runs_Scored) AS runs_conceded
  FROM bowling_balls bb
  JOIN Batsman_Scored bs
    ON bs.Match_Id=bb.Match_Id
   AND bs.Over_Id=bb.Over_Id
   AND bs.Ball_Id=bb.Ball_Id
   AND bs.Innings_No=bb.Innings_No
  GROUP BY bb.Bowler, bb.Match_Id
),
wickets_by_bowler AS (
  SELECT 
    bb.Bowler AS bowler_id,
    bb.Match_Id,
    COUNT(*) AS wickets
  FROM bowling_balls bb
  JOIN Wicket_Taken w
    ON w.Match_Id=bb.Match_Id
   AND w.Over_Id=bb.Over_Id
   AND w.Ball_Id=bb.Ball_Id
   AND w.Innings_No=bb.Innings_No
  JOIN Out_Type ot ON ot.Out_Id = w.Kind_Out
  WHERE ot.Out_Name IN ('caught','bowled','lbw','stumped','caught and bowled','hit wicket')
  GROUP BY bb.Bowler, bb.Match_Id
),
bowling_figures AS (
  SELECT
    p.Player_Id      AS bowler_id,
    p.Player_Name,
    rc.Match_Id,
    COALESCE(wk.wickets, 0)     AS wickets,
    COALESCE(rc.runs_conceded,0) AS runs_conceded
  FROM runs_conceded rc
  LEFT JOIN wickets_by_bowler wk
    ON wk.bowler_id=rc.bowler_id AND wk.Match_Id=rc.Match_Id
  JOIN Player p ON p.Player_Id=rc.bowler_id
),
ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY bowler_id ORDER BY wickets DESC, runs_conceded ASC) AS rn
  FROM bowling_figures
)
SELECT bowler_id,
       Player_Name,
       Match_Id,
       wickets,
       runs_conceded
FROM ranked
WHERE rn=1
ORDER BY wickets DESC, runs_conceded ASC, Player_Name;

'''

df_sql = pd.read_sql_query(sql,con=conn)
df_sql.head()

,bowler_id,Player_Name,Match_Id,wickets,runs_conceded
0,102,Sohail Tanvir,336010,6,14
1,430,A Zampa,980984,6,19
2,124,A Kumble,392187,5,5
3,84,I Sharma,501234,5,11
4,194,SL Malinga,501206,5,12


6. Query 5: Comprehensive Career Metrics

Combine all the previous chunks into a single comprehensive query to get detailed career metrics for players.


In [21]:
sql='''
WITH matches_played AS (
  SELECT pm.Player_Id, COUNT(DISTINCT pm.Match_Id) AS matches
  FROM Player_Match pm
  GROUP BY pm.Player_Id
),
batsman_runs AS (
  SELECT p.Player_Id, SUM(bs.Runs_Scored) AS total_runs
  FROM Ball_by_Ball b
  JOIN Batsman_Scored bs
    ON bs.Match_Id=b.Match_Id
   AND bs.Over_Id=b.Over_Id
   AND bs.Ball_Id=b.Ball_Id
   AND bs.Innings_No=b.Innings_No
  JOIN Player p ON p.Player_Id=b.Striker
  GROUP BY p.Player_Id
),
batsman_match_runs AS (
  SELECT p.Player_Id, b.Match_Id, SUM(bs.Runs_Scored) AS runs_in_match
  FROM Ball_by_Ball b
  JOIN Batsman_Scored bs
    ON bs.Match_Id=b.Match_Id
   AND bs.Over_Id=b.Over_Id
   AND bs.Ball_Id=b.Ball_Id
   AND bs.Innings_No=b.Innings_No
  JOIN Player p ON p.Player_Id=b.Striker
  GROUP BY p.Player_Id, b.Match_Id
),
fifties_hundreds AS (
  SELECT Player_Id,
         SUM(CASE WHEN runs_in_match BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS fifties,
         SUM(CASE WHEN runs_in_match >= 100 THEN 1 ELSE 0 END)           AS hundreds
  FROM batsman_match_runs
  GROUP BY Player_Id
),
runs_conceded AS (
  SELECT bb.Bowler AS bowler_id, SUM(bs.Runs_Scored) AS runs_conceded
  FROM Ball_by_Ball bb
  JOIN Batsman_Scored bs
    ON bs.Match_Id=bb.Match_Id
   AND bs.Over_Id=bb.Over_Id
   AND bs.Ball_Id=bb.Ball_Id
   AND bs.Innings_No=bb.Innings_No
  GROUP BY bb.Bowler
),
wickets_by_bowler AS (
  SELECT bb.Bowler AS bowler_id, COUNT(*) AS wickets
  FROM Ball_by_Ball bb
  JOIN Wicket_Taken w
    ON w.Match_Id=bb.Match_Id
   AND w.Over_Id=bb.Over_Id
   AND w.Ball_Id=bb.Ball_Id
   AND w.Innings_No=bb.Innings_No
  JOIN Out_Type ot ON ot.Out_Id = w.Kind_Out
  WHERE ot.Out_Name IN ('caught','bowled','lbw','stumped','caught and bowled','hit wicket')
  GROUP BY bb.Bowler
),
best_bowling_per_match AS (
  WITH rc AS (
    SELECT bb.Bowler AS bowler_id, bb.Match_Id, SUM(bs.Runs_Scored) AS runs_conceded
    FROM Ball_by_Ball bb
    JOIN Batsman_Scored bs
      ON bs.Match_Id=bb.Match_Id
     AND bs.Over_Id=bb.Over_Id
     AND bs.Ball_Id=bb.Ball_Id
     AND bs.Innings_No=bb.Innings_No
    GROUP BY bb.Bowler, bb.Match_Id
  ),
  wk AS (
    SELECT bb.Bowler AS bowler_id, bb.Match_Id, COUNT(*) AS wickets
    FROM Ball_by_Ball bb
    JOIN Wicket_Taken w
      ON w.Match_Id=bb.Match_Id
     AND w.Over_Id=bb.Over_Id
     AND w.Ball_Id=bb.Ball_Id
     AND w.Innings_No=bb.Innings_No
    JOIN Out_Type ot ON ot.Out_Id = w.Kind_Out
    WHERE ot.Out_Name IN ('caught','bowled','lbw','stumped','caught and bowled','hit wicket')
    GROUP BY bb.Bowler, bb.Match_Id
  ),
  bf AS (
    SELECT rc.bowler_id, rc.Match_Id, COALESCE(wk.wickets,0) AS wickets, rc.runs_conceded
    FROM rc LEFT JOIN wk
      ON wk.bowler_id=rc.bowler_id AND wk.Match_Id=rc.Match_Id
  )
  SELECT bowler_id, wickets, runs_conceded,
         ROW_NUMBER() OVER (PARTITION BY bowler_id ORDER BY wickets DESC, runs_conceded ASC) AS rn
  FROM bf
),
best_bowling AS (
  SELECT bowler_id, wickets AS best_wkts, runs_conceded AS best_runs
  FROM best_bowling_per_match
  WHERE rn=1
)
SELECT
  p.Player_Id,
  p.Player_Name,
  mp.matches,
  COALESCE(br.total_runs,0) AS batting_runs,
  COALESCE(fh.fifties,0)    AS fifties,
  COALESCE(fh.hundreds,0)   AS hundreds,
  COALESCE(wb.wickets,0)    AS bowling_wickets,
  COALESCE(bb.best_wkts,0)  AS best_bowling_wickets,
  COALESCE(bb.best_runs,0)  AS best_bowling_runs
FROM Player p
JOIN matches_played    mp ON mp.Player_Id = p.Player_Id
LEFT JOIN batsman_runs br ON br.Player_Id = p.Player_Id
LEFT JOIN fifties_hundreds fh ON fh.Player_Id = p.Player_Id
LEFT JOIN wickets_by_bowler wb ON wb.bowler_id = p.Player_Id
LEFT JOIN best_bowling      bb ON bb.bowler_id = p.Player_Id
ORDER BY batting_runs DESC, bowling_wickets DESC, p.Player_Name;

'''

df_sql = pd.read_sql_query(sql,con=conn)
df_sql.head()

,Player_Id,Player_Name,matches,batting_runs,fifties,hundreds,bowling_wickets,best_bowling_wickets,best_bowling_runs
0,21,SK Raina,146,4106,28,1,24,2,0
1,8,V Kohli,138,4105,27,4,4,2,23
2,57,RG Sharma,142,3874,29,1,15,4,6
3,40,G Gambhir,132,3634,31,0,0,0,0
4,162,CH Gayle,91,3447,21,5,18,3,21
